In [ ]:
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.weightstats as st
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

# 1. Regressive Tennis T-Test

Use a linear regression and statsmodels to run a t-test on whether Federer scores more points than his opponents in the `tennis.csv` dataset.

Give a one-paragraph interpretation of the coefficient, and the meaning of the p-value. 

Also answer the following: should your regression include a constant term? Why or why not? How would it change the interpretation of your coefficient and p-value?

In [ ]:
df = pd.read_csv(r'C:\Users\David\Documents\code\Module 3\m3-2-regression-interpretation\data\tennis.csv')
df = df.dropna()
df = df.sort_values(by='player1 total points won')

y = df[['player1 total points won']]
x = df[['player2 total points won']]
X = sm.add_constant(x)

est = sm.OLS(y, X).fit(cov_type='HC2')
print(est.summary())

## In this instance, the coefficient denotes for each point Federrer scores, how many more points his opponent scores (on average). The p-value denotes that as a t-test, we can reject the null hypothesis that there is no difference in the means of the two variables.

## The constant is essential in creating an accurate coefficient. Otherwise, it would measure the slope from a 0 intercept, which is not an expected score in professional tennis.

# 2. College admissions

Using the `college.csv` dataset, answer the following:

1. Is the relation between `Top10perc` and `Top25perc` best fit using a model with only one variable, or one variable and a polynomial of degree 2? Is a constant term useful? How would you select for the best of these model specifications?

2. Do private schools see more admissions overall? T-test this using a linear regression. Hint: use a binary explanatory variable for `Private`. Explain your model specification choices.


In [ ]:
df = pd.read_csv(r'C:\Users\David\Documents\code\Module 3\m3-2-regression-interpretation\data\college.csv')
df['Private'] = df.Private.replace(('Yes', 'No'), (1, 0))

##1.
df  = df.sort_values(by='Top25perc')
y = df['Top10perc']
x = df[['Top25perc']].copy()
x['Top25perc_sq'] = x['Top25perc']**2

X = sm.add_constant(x)

bad_model = sm.OLS(y,x).fit()
model = sm.OLS(y, X[['Top25perc', 'const']]).fit()
poly_model = sm.OLS(y, X).fit(cov_type='HC2')

# print(bad_model.summary())
# print(model.summary())
# print(poly_model.summary())

## On analyzing the AIC,BIC, and R-squared values, it is clear that including a polynomial feature improves the model. Without the constant, our AIC and BIC become much greater.

##2.
df = df.sort_values(by='Private')
y = df[['Accept']]
x = df[['Private']]
model = sm.OLS(y, x).fit()

# model.summary()

## In this case I had to leave out the constant, as we're working on a binary (0,1) and adding a constant would distort the trend we are observing between two variables.

##What we are seeing is a coefficient of 1305.7, meaning that as we move from non-private (0) to Private (1), we see an average increase of just over 1000 acceptances.


# 3. Auto prediction

Using the `auto.csv` dataset, perform a simple linear regression with `mpg` as the response variable and horsepower as the predictor. Answer the following:

 i. Is there a relationship between the predictor and the response?
 
 ii. How strong is the relationship between the predictor and the response?
 
 iii. Is the relationship between the predictor and the response positive or negative?

 iv. What is the predicted mpg associated with a horsepower of 98? What are the associated 95 % confidence and prediction intervals ?

 v. Make a regression plot the response and the predictor.

In [ ]:
#First, I need to clean this dataset

df = pd.read_csv(r'C:\Users\David\Documents\code\Module 3\m3-2-regression-interpretation\data\auto.csv')
columns_ = df.columns.str.split()

df = df.rename(columns={'mpg\tcylinders\tdisplacement\thorsepower weight\tacceleration\tyear\torigin\tname': 1})
df = df.drop([0])

hold = pd.DataFrame()

counter = 0 

for i in range(0, len(columns_[0])):
    df[1] = df[1].str.replace("   ", ' ')
    df[1] = df[1].str.replace("  ", ' ')
    df[1] = df[1].str.replace( '1\t"', ' ')
    df[1] = df[1].str.replace( '2\t"', ' ')
    df[1] = df[1].str.replace( '3\t"', ' ')
    df[1] = df[1].str.replace( '?', '')
    df = df[1].str.split(" ", n = 1, expand = True) 
    hold[counter] = df[0]
    df = df.drop([0], axis=1)

    if counter <= 5:
        hold[counter] = hold[counter].astype(float)
    counter += 1

df = pd.DataFrame(data=hold)

df = df.rename(columns= {0: 'mpg' ,1:'cylinders',2:'displacement',3:'horsepower',4:'weight',5:'acceleration',6:'year',7:'origin',8:'name'})


In [ ]:
df = df.sort_values(by='horsepower')
df = df.drop(df.tail(5).index) #There are some serious outliers that need to be dropped

y = df.mpg
x = df[['horsepower']]
x['horsepower_sq'] = x['horsepower'] ** 2
X = sm.add_constant(x)

model = sm.OLS(y, X).fit(cov_type='HC2')
model.summary()


## i. Is there a relationship between the predictor and the response?
## Yes, the R-squared value of 0.688 denotes that our model matches at least some predictability

## ii. How strong is the relationship between the predictor and the response?
## It is alright, although I would not depict it as 'strong'. We can only

## iii. Is the relationship between the predictor and the response positive or negative?
## Positive (see coefficient).

## iv. What is the predicted mpg associated with a horsepower of 98? What are the associated 95 % confidence and prediction intervals ?
predictions = model.predict(X)
print('The predicted mpg when horsepower is 98:', predictions.loc[predictions.index == 98].iloc[0])

## v. Make a regression plot the response and the predictor.
yfit = est.predict(X)
fig, ax = plt.subplots()
ax.scatter(x['horsepower'], y, color='purple', alpha=0.4)
ax.plot(x['horsepower'], yfit, color='blue', alpha=0.8, linewidth=3)

plt.show()

# 4. Auto Multiple Regression

Perform a multiple linear regression with `mpg` as the response and all other variables except name in `auto` as the predictors. Comment on the output:

i. Is there a relationship between the predictors and the response?

ii. Which predictors appear to have a statistically significant relationship to the response?

iii. What does the coefficient for the year variable suggest?

iv. Comment on any problems you see with the fit. Do the residual plots suggest any unusually large outliers?

v. Is there heteroscedasticity in the fit?

In [ ]:
df_ = df.copy()
df_.year = df_.year.astype(int)

y = df_.mpg
x = df_.drop(['mpg', 'origin', 'name'], axis=1)

X = sm.add_constant(x)

model = sm.OLS(y, X).fit(cov_type='HC2')
model.summary()

## i. 
## For most of them, yes. The R-squared value denotes the existence of a relationship.

## ii. 
## In reding the p-values, it appears that only horsepower, year and weight have statistically significant relationships with mpg.

## iii. 
## This suggests that as the year increases by 1, the mpg increases by 0.7526.

## iv. 
## The outliers in horsepower are a major problem which were immediately observeable in the residuals plot. I had to remove them in the last exercise since they seemed to ignore the very noticeable trend with mpg and skew the whole plot. Because the difference in values was so great, I felt it was fair to remove them.

## v. 
## Yes there is. THis is indicated under the warnings as well as 'Covariance Type: HC2'.


# 5. Car Seats

This question should be answered using the Carseats data set

1. Fit a multiple regression model to predict Sales using Price, Urban, and US.

2. Provide an interpretation of each coefficient in the model. Be careful—some of the variables in the model are qualitative!

3. For which of the predictors can you reject the null hypothesis H0 : βj = 0?

4. On the basis of your response to the previous question, fit a smaller model that only uses the predictors for which there is evidence of association with the outcome.

5. How well do the models in 1 and 4 fit the data? Explain which statistics show the difference.

6. Using the model from (e), obtain 95 % confidence intervals for the coefficient(s).

In [385]:
## 1.
df = pd.read_csv(r'C:\Users\David\Documents\code\Module 3\m3-2-regression-interpretation\data\carseats.csv')

df['Urban'] = df.Urban.replace(('Yes', 'No'), (1, 0))
df['US'] = df.US.replace(('Yes', 'No'), (1, 0))

y = df['Sales']
x = df[['Price', 'Urban', 'US']]

X = sm.add_constant(x)

model = sm.OLS(y, X).fit(cov_type='HC2')
# print(model.summary())

## 2. 
## The negative coefficient for Price demonstrates that Sales decrease as the Price increases. The coefficient for Urban shows a slight decrease in Sales in urban spaces.

## 3.
## All except for Urban, which has a large p-value.

## 4. On the basis of your response to the previous question, fit a smaller model that only uses the predictors for which there is evidence of association with the outcome.
y_ = df['Sales']
x_ = df[['Price', 'US']]

X_ = sm.add_constant(x_)

model_ = sm.OLS(y_, X_).fit(cov_type='HC2')
model_.summary()
## How well do the models in 1 and 4 fit the data? Explain which statistics show the difference.
## The model fits fairly well. Although the low R-Squared value suggests that the observations are far from the model, the low standard errors suggest that the fit is farily accurate. The AIC and BIC also aren't terribly high.

## Using the model from (e), obtain 95 % confidence intervals for the coefficient(s).
model_.conf_int(alpha=0.05, cols=None)

,0,1
const,11.811364,14.250221
Price,-0.064605,-0.044350
US,0.716136,1.683150
